# Supply Chain Optimization

Supply Chain Optimization: An Overview


In this section, we're diving into the world of supply chain optimization. Imagine you're running a company that makes a popular product. To get this product to your customers, you need to figure out the best way to produce and ship it. This involves deciding how much to make, where to make it, and how to get it to the distribution centers.

Supply chain optimization helps us answer these questions in the most cost-effective way. We use math and computer programs to find the best solutions, considering factors like production costs, shipping costs, factory capacities, and customer demand.

Our goal is simple: minimize costs while meeting all our customers' needs. This involves a lot of number crunching and problem-solving, but don't worry, we'll walk through it step by step. By the end of this, you'll see how optimizing your supply chain can save money and improve efficiency.

## Understanding Our Supply Chain Scenario

In our scenario, we're managing a company that produces a single product in three different factories, labeled as Factory A, B, and C. Each of these factories has its own production capabilities and costs associated with making the product. Our task is to distribute this product to four distinct distribution centers, known as DC1, DC2, DC3, and DC4, each with its own demand that we must fulfill.

The challenge here is to figure out how much each factory should produce and how the products should be shipped to the distribution centers in the most cost-effective manner. This involves considering the production costs at each factory, the shipping costs from factories to distribution centers, and ensuring that the demand at each distribution center is met precisely.

Our objective is clear: minimize the total cost of production and shipping. This means we need to make smart decisions about where to produce more and where to cut back, as well as how to route our shipments to satisfy demand at the lowest possible cost.

Through this scenario, we'll explore the complexities of supply chain management and develop strategies to optimize our operations. It's a practical exercise that will show you the importance of efficient planning and execution in meeting customer needs and managing costs.

## Mathematical Model for Supply Chain Optimization

### Indices:
- $i$: Factories ($i \in \{A, B, C\}$)
- $j$: Distribution Centers ($j \in \{1, 2, 3, 4\}$)

### Parameters:
- $C_i$: Production capacity of factory $i$
- $P_i$: Production cost per unit at factory $i$
- $D_j$: Demand at distribution center $j$
- $S_{ij}$: Shipping cost per unit from factory $i$ to distribution center $j$

### Decision Variables:
- $X_{ij}$: Number of units to ship from factory $i$ to distribution center $j$
- $Y_i$: Number of units produced at factory $i$

### Objective Function:
Minimize the total cost of production and shipping:
$$
\text{Minimize } Z = \sum_{i} \sum_{j} (P_i + S_{ij}) \cdot X_{ij}
$$

### Constraints:
1. **Production Capacity**: The production at each factory cannot exceed its capacity.
   $$
   Y_i \leq C_i \quad \forall i
   $$
2. **Demand Satisfaction**: The total shipments to each distribution center must meet its demand.
   $$
   \sum_{i} X_{ij} = D_j \quad \forall j
   $$
3. **Production-Shipping Balance**: The total produced at each factory must equal the total shipped from that factory.
   $$
   \sum_{j} X_{ij} = Y_i \quad \forall i
   $$
4. **Non-negativity**: We cannot produce or ship negative quantities.
   $$
   X_{ij} \geq 0, Y_i \geq 0 \quad \forall i, j
   $$


In [1]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("SupplyChainOptimization")

# Parameters
factories = ['A', 'B', 'C']
distribution_centers = [1, 2, 3, 4]
C = {'A': 100, 'B': 150, 'C': 200}
P = {'A': 5, 'B': 4, 'C': 6}
D = {1: 80, 2: 65, 3: 70, 4: 85}
S = {('A', 1): 2, ('A', 2): 3, ('A', 3): 2.5, ('A', 4): 3.5,
     ('B', 1): 2.5, ('B', 2): 2, ('B', 3): 3, ('B', 4): 2.5,
     ('C', 1): 3, ('C', 2): 3.5, ('C', 3): 2, ('C', 4): 2.5}

# Decision Variables
X = model.addVars(factories, distribution_centers, name="X", vtype=GRB.CONTINUOUS)
Y = model.addVars(factories, name="Y", vtype=GRB.CONTINUOUS)

# Objective Function
model.setObjective(gp.quicksum((P[i] + S[i, j]) * X[i, j] for i in factories for j in distribution_centers), GRB.MINIMIZE)

# Constraints
model.addConstrs(Y[i] <= C[i] for i in factories)
model.addConstrs(gp.quicksum(X[i, j] for i in factories) == D[j] for j in distribution_centers)
model.addConstrs(gp.quicksum(X[i, j] for j in distribution_centers) == Y[i] for i in factories)
model.addConstrs(X[i, j] >= 0 for i in factories for j in distribution_centers)

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Objective: {model.ObjVal}")
    for v in model.getVars():
        print(f"{v.VarName}: {v.X}")

    # Print Gurobi report
    model.printAttr('X')
    model.printQuality()
else:
    print("No optimal solution found.")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 15 columns and 42 nonzeros
Model fingerprint: 0xc3f8605e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 15 rows and 3 columns
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9525000e+03   1.875000e+01   0.000000e+00      0s
       3    2.0525000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.052500000e+03
Optimal Objective: 2052.5
X[A,1]: 80.0
X[A,2]: 0.0
X[A,3]: 20.0
X[A,4]: 0.0
X[B,1]: 0.0
X[B,2]: 65.0
X[B,3]: 0